In [1]:
import os
%pip install pymysql
import pymysql
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def connect_db(user:str, user_pwd:str, db:str, host:str) -> pd.DataFrame:
    conn = pymysql.connect(
        host=host,
        port=3306,
        user=user,
        passwd=user_pwd,
        db=db)

    df = pd.read_sql_query("SELECT * FROM final_project.books",
    conn)
    df_copy = df.copy()
    df_copy = df_copy.drop(['id'], axis=1)
    conn.close()
    return df_copy

df_copy = connect_db('mysql_user','SxEmnfzY94$55eBQ', 'final_project', 'localhost')


Note: you may need to restart the kernel to use updated packages.


In [129]:
%pip show sklearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    genre_df_dummies = pd.get_dummies(df['genres'])
    df_copy_cleaned = df.drop(['author', 'description', 'language', 'coverImg', 'likedPercent', 'isbn', 'title', 'genres'], axis=1)
    df_copy_merged = pd.concat([genre_df_dummies, df_copy_cleaned], axis=1)
    return df_copy_merged

In [3]:
X = preprocess_data(df_copy)
X

,comedy,environment,fantasy,fiction,food,health,horror,literature,nonfiction,romance,tech,thriller,war,rating,numRatings
0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.33,6376780.0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,4.50,2507620.0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,4.28,4501080.0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,4.26,2998240.0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,3.60,4964520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7238,1,0,0,0,0,0,0,0,0,0,0,0,0,4.23,44034.0
7239,0,0,0,0,0,0,0,0,0,0,0,1,0,3.76,25143.0
7240,0,0,0,1,0,0,0,0,0,0,0,0,0,3.50,27511.0
7241,0,0,0,0,0,0,0,0,0,1,0,0,0,4.04,24431.0


In [4]:
def normalize_data(data:pd.DataFrame) -> pd.DataFrame:
    min_max_scaler = MinMaxScaler()
    features_encoded = min_max_scaler.fit_transform(data)
    return features_encoded

def generate_df_for_training(encoded_data: np.ndarray):
    df_kmeans = pd.DataFrame(encoded_data, columns = ['comedy', 'environment', 'fantasy', 'fiction', 'health', 'horror',
           'literature', 'nonfiction', 'romance', 'tech', 'food', 'thriller', 'war',
           'rating', 'numRatings'])
    return df_kmeans

encoded_data = normalize_data(X)
df_kmeans = generate_df_for_training(encoded_data)


In [5]:
def kmeans_model_elaboration(data:pd.DataFrame):
    kmeans = KMeans(n_clusters=5, random_state=0).fit(data)
    return kmeans

In [6]:
kmeans = kmeans_model_elaboration(encoded_data)

In [7]:
def predict(data:pd.DataFrame, df_kmeans: pd.DataFrame ,encoded_data: pd.DataFrame, kmeans) -> pd.DataFrame:
    data['category'] = kmeans.predict(encoded_data)
    df_kmeans['category'] = kmeans.predict(encoded_data)
    return data

In [8]:
category_predictions = predict(df_copy, df_kmeans, encoded_data, kmeans)

In [9]:
def nearest_neighbors_modelisation(data:pd.DataFrame):
    model = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
    model.fit(data)
    dist, idlist = model.kneighbors(data)
    return dist, idlist

In [10]:
dist, idlist = nearest_neighbors_modelisation(df_kmeans)

In [11]:
def book_recommendation_engine(book_name):
    book_list_name = []
    book_list_id = []
    book_id = df_copy[df_copy['title'].str.contains(book_name, case=False)].index
    book_id = book_id[0]
    for newid in idlist[book_id]:
        book_list_name.append(df_copy.loc[newid].title)
#     return book_list_name
    return df_copy.iloc[idlist[book_id]]

In [12]:
book_list_name = book_recommendation_engine('American Sniper: The Autobiography of the Most Lethal Sniper in U.S. Military History')
book_list_name

,title,rating,author,description,language,isbn,genres,numRatings,likedPercent,coverImg,category
2945,American Sniper: The Autobiography of the Most...,4.00,Chris Kyle,"From 1999 to 2009, U.S. Navy SEAL Chris Kyle r...",English,9780062107060,nonfiction,110501.0,91.0,https://i.gr-assets.com/images/S/compressed.ph...,4
6015,Benjamin Franklin: An American Life,4.00,Walter Isaacson (Goodreads Author),Benjamin Franklin is the Founding Father who w...,English,9780743258074,nonfiction,113854.0,91.0,https://i.gr-assets.com/images/S/compressed.ph...,4
4739,The War of Art: Break Through the Blocks & Win...,4.00,Steven Pressfield (Goodreads Author),Internationally bestselling author of Last of ...,English,9780446691437,nonfiction,72969.0,90.0,https://i.gr-assets.com/images/S/compressed.ph...,4
1943,The Tao of Pooh,3.99,Benjamin Hoff,The Wisdom of Pooh.Is there such thing as a We...,English,9781405204262,nonfiction,101520.0,92.0,https://i.gr-assets.com/images/S/compressed.ph...,4
5208,"SuperFreakonomics: Global Cooling, Patriotic P...",3.99,Steven D. Levitt,The New York Times best-selling Freakonomics w...,English,9780060889579,nonfiction,121251.0,94.0,https://i.gr-assets.com/images/S/compressed.ph...,4
3260,Getting Things Done: The Art of Stress-Free Pr...,3.99,David Allen,"In today's world, yesterday's methods just don...",English,9780142000281,nonfiction,128554.0,91.0,https://i.gr-assets.com/images/S/compressed.ph...,4
1402,Beyond Good and Evil,4.00,Friedrich Nietzsche,Friedrich Nietzsche's Beyond Good and Evil is ...,English,9780140449235,nonfiction,66727.0,92.0,https://i.gr-assets.com/images/S/compressed.ph...,4
3533,Under the Banner of Heaven,4.00,Jon Krakauer (Goodreads Author),"A Story of Violent FaithA multilayered, bone-c...",English,9780330419123,nonfiction,156956.0,94.0,https://i.gr-assets.com/images/S/compressed.ph...,4
1332,The Diving Bell and the Butterfly,4.00,Jean-Dominique Bauby,‘Locked-in syndrome: paralysed from head to to...,English,9780375701214,nonfiction,59577.0,93.0,https://i.gr-assets.com/images/S/compressed.ph...,4
4940,Escape from Camp 14: One Man's Remarkable Odys...,4.00,Blaine Harden (Goodreads Author),"A New York Times bestseller, the shocking stor...",English,9780670023325,nonfiction,58150.0,95.0,https://i.gr-assets.com/images/S/compressed.ph...,4


In [54]:
df_copy.index +=1

In [126]:
from sqlalchemy import create_engine
def insert_to_db(user:str, user_pwd:str, db:str, host:str, df:pd.DataFrame):
    sqlEngine       = create_engine(f"mysql+pymysql://{user}:{user_pwd}@{host}/{db}?charset=utf8mb4")
    dbConnection    = sqlEngine.connect()
    df.to_sql(name='books', con = dbConnection, if_exists = 'replace', chunksize = 1000)
    

In [127]:
insert_to_db('mysql_user','SxEmnfzY94$55eBQ', 'final_project', 'localhost', df_copy)
